In [1]:
from os.path import join
from scamp import test_run, Session, settings, wait
from config import ARTIFACTS_RELPATH
from utils.scamp import set_default_scamp_settings
from utils.supercollider import SuperCollider
from scamp_extensions.pitch import Scale, ScaleType, hertz_to_midi
from numbers import Real
from operator import itemgetter
from itertools import chain
import numpy as np
from config import ARTIFACTS_RELPATH

set_default_scamp_settings()

Testing for working audio driver...
jackdmp 1.9.22
Copyright 2001-2005 Paul Davis and others.
Copyright 2004-2016 Grame.
Copyright 2016-2023 Filipe Coelho.
jackdmp comes with ABSOLUTELY NO WARRANTY
This is free software, and you are welcome to redistribute it
under certain conditions; see the file COPYING for details


ALSA lib conf.c:4005:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/opt/conda/envs/default/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default
fluidsynth: error: Failed to open the "default" audio device
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'coreaudio'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseaudio, sdl2
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'dsound'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseau

In [3]:
HOST = "supercollider"
session = Session().run_as_server()
session.tempo = 65

supercollider = SuperCollider(
    join(ARTIFACTS_RELPATH, "scd_runtime.json"), HOST
)
connection_params = {
    "port": supercollider.get_lang_port(),
    "ip_address": HOST,
}
inst1 = session.new_osc_part(
    "scamp1", port=supercollider.get_lang_port(), ip_address=HOST
)
inst2 = session.new_osc_part(
    "scamp1", port=supercollider.get_lang_port(), ip_address=HOST
)

In [3]:
def bach_bwv_147_10():
    # fmt: off
    # JS Bach, BWV 147 No.10 - Jesu, Joy of Man's Desiring
    bach_bwv_147_10_sequence = np.array( [
            [ 0, 2, 4, 7, 5, 5, 9, 7, 7, 12, 11, 12, 7, 4, 0, 2, 4, ],
            # [ 5, 7, 9, 7, 5, 4, 2, 4, 0, -1, 0, 2, -5, -1, 2, 5, 4, 2, 4, ],
            # [ 0, 2, 4, 7, 5, 5, 9, 7, 7, 12, 11, 12, 7, 4, ],
            # [ 0, 2, 4, -5, 7, 5, 4, 2, 0, -5, 0, -1, 0, 4, 7, 12, 7, 4, ],
        ],
        dtype=object,
    )
    # fmt: on
    sequence = [
        48 + i for i in np.hstack(bach_bwv_147_10_sequence).tolist()
    ] * 16
    velocities = np.repeat(0.8, len(sequence)).tolist()
    durations = np.repeat(1 / 3, len(sequence)).tolist()

    for pitch, velocity, duration in zip(sequence, velocities, durations):
        inst1.play_note(
            pitch,
            velocity,
            duration,
            [{"articulations": "staccato"}],
        )
        # wait(duration)

In [4]:
def some_bass_line(note_duration: float):
    sequence = [48 + i for i in [0, 2, 4]] * 6
    velocities = np.repeat(0.8, len(sequence)).tolist()
    durations = np.repeat(note_duration, len(sequence)).tolist()

    for pitch, velocity, duration in zip(sequence, velocities, durations):
        inst2.play_note(
            pitch,
            velocity,
            duration,
            [{"articulations": "staccato"}, "pitch - 12"],
        )
        # wait(duration)

In [7]:
session.fork(bach_bwv_147_10)
session.fork(some_bass_line, args=[1 / 2])
session.wait_for_children_to_finish()

ValueError: list.remove(x): x not in list

In [9]:
session.fast_forward(1000)
# sequence = [0, 5, 7, 13, 5, 7, 1] * 4
# sequence = [48 + i for i in sequence]
# for root in sequence:
#     chord = [root + i for i in range(0, 13, 2)]
#     inst.play_chord(chord, 0.8, 2)
#     wait(2)